In [7]:
include("ellipsoid2xyz.jl");
include("xyz2ellh.jl");

In [8]:
lat_1 = (56 + 33/60 + 44.4528/(60*60)) * pi/180
lon_1 = (8 + 18/60 + 24.865/(60*60)) * pi/180
height_1 = 43.282;

point_test = ellipsoid2xyz(lat_1, lon_1, height_1)

(3.485824828590261e6, 508956.0100922042, 5.299237309602344e6)

In [9]:
coords_test = xyz2ellh(point_test...)


(0.9871992052588295, 0.14498287683733727, 43.28199999965727)

In [10]:
coords_test .- ( lat_1, lon_1, height_1)

(0.0, 0.0, -3.427231831665267e-10)

In [12]:
llh = Array{Float64}(undef,2,3)

2×3 Array{Float64,2}:
 0.0  0.0  0.0
 0.0  0.0  0.0

In [15]:
llh[1,:] .= coords_test

3-element view(::Array{Float64,2}, 1, :) with eltype Float64:
  0.9871992052588295 
  0.14498287683733727
 43.28199999965727   

In [17]:
 coords_test

(0.9871992052588295, 0.14498287683733727, 43.28199999965727)

In [16]:
llh[1,:]

3-element Array{Float64,1}:
  0.9871992052588295 
  0.14498287683733727
 43.28199999965727   

In [8]:
atan(1,-1)*180/pi

135.0

In [10]:
sin(2)

0.9092974268256817

In [17]:
function xyz2ell2(X,Y,Z,semi_major_axis=6378137.,flattening=1/298.257223563)
    e2 = flattening*(2-flattening)
    
    
    elat=1.e-12
    eht=1.e-5
    p=sqrt(X^2+Y^2)
    lat=atan(Z,p./(1-e2))
    h=0
    dh=1
    dlat=1
    
    
    while (dlat>elat) | (dh>eht)
      lat0=lat
      h0=h
      v=semi_major_axis/sqrt(1-e2*sin(lat)*sin(lat))
      h=p*cos(lat)+Z*sin(lat)-(semi_major_axis^2)/v  # Bowring formula
      lat=atan(Z, p*(1-e2*v/(v+h)))
      dlat=abs(lat-lat0)
      dh=abs(h-h0)
    end
    lon=atan(Y,X)
    return lat, lon, h
end

xyz2ell2 (generic function with 3 methods)

In [19]:
xyz2ell2(point_test...) .- ( lat_1, lon_1, height_1)

(0.0, 0.0, -3.427231831665267e-10)

In [ ]:
function [lat,lon,h]=xyz2ell2(X,Y,Z,a,e2)
% XYZ2ELL2  Converts cartesian coordinates to ellipsoidal.
%   Uses iterative alogithm with Bowring height formula
%   to improve convergence rate (see B.R. Bowring, "The
%   accuracy of geodetic latitude and height equations",
%   Survey Review, v28 #218, October 1985 pp.202-206).
%   Computation time is only slightly worse than for the
%   Bowring direct formula but accuracy is better.  Vectorized.
%   See also XYZ2ELL, XYZ2ELL2.
% Version: 2011-02-19
% Useage:  [lat,lon,h]=xyz2ell2(X,Y,Z,a,e2)
%          [lat,lon,h]=xyz2ell2(X,Y,Z)
% Input:   X \
%          Y  > vectors of cartesian coordinates in CT system (m)
%          Z /
%          a   - ref. ellipsoid major semi-axis (m); default GRS80
%          e2  - ref. ellipsoid eccentricity squared; default GRS80
% Output:  lat - vector of ellipsoidal latitudes (radians)
%          lon - vector of ellipsoidal longitudes (radians)
%          h   - vector of ellipsoidal heights (m)

% Copyright (c) 2011, Michael R. Craymer
% All rights reserved.
% Email: mike@craymer.com

if nargin ~= 3 & nargin ~= 5
  warning('Incorrect number of input arguments');
  return
end
if nargin == 3
  [a,b,e2]=refell('grs80');
end

elat=1.e-12;
eht=1.e-5;
p=sqrt(X.*X+Y.*Y);
lat=atan2(Z,p./(1-e2));
h=0;
dh=1;
dlat=1;
while sum(dlat>elat) | sum(dh>eht)
  lat0=lat;
  h0=h;
  v=a./sqrt(1-e2.*sin(lat).*sin(lat));
  h=p.*cos(lat)+Z.*sin(lat)-(a*a)./v;  % Bowring formula
  lat=atan2(Z, p.*(1-e2.*v./(v+h)));
  dlat=abs(lat-lat0);
  dh=abs(h-h0);
end
lon=atan2(Y,X);